# IMPORT PACKEGES

In [ ]:
import numpy as np
import h5py
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import ast

# PLOT SETTINGS

In [ ]:
# Font settings
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=True)

# Tick settings
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4

mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2

# Axis linewidth
mpl.rcParams['axes.linewidth'] = 2

# Tick direction and enabling ticks on all sides
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

# Function to apply custom tick locators and other settings to an Axes object
def apply_custom_settings(ax, leg, log_scale_y=False):

    if log_scale_y:
        # Use LogLocator for the y-axis if it's in log scale
        ax.set_yscale('log')
        ax.yaxis.set_major_locator(LogLocator(base=10.0))
        ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto', numticks=100))
    else:
        # Use AutoLocator for regular scales
        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Apply the AutoLocator for the x-axis
    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    
    # Legend settings
    leg.get_frame().set_edgecolor('w')
    leg.get_frame().set_linewidth(0.0)

# PHYSICAL CONSTANTS


In [ ]:
class CGSUnitsConst:
    eV = 1.60218e-12  # erg

class PhysConst:
    c = 2.99792458e10  # cm/s
    c2 = c * c
    c4 = c2 * c2
    hbar = 1.05457266e-27  # erg s
    hbarc = hbar * c  # erg cm
    hc = hbarc*2*np.pi
    GF = (1.1663787e-5 / (1e9 * 1e9 * CGSUnitsConst.eV * CGSUnitsConst.eV))
    Mp = 1.6726219e-24  # g
    sin2thetaW = 0.23122
    kB = 1.380658e-16  # erg/K
    G = 6.67430e-8 # cm3 g−1 s-2
    Msun = 1.9891e33 # g

# DIRECTORY PATH

In [ ]:
###################################################
# SEQUENTIAL INSTABILITY
directory_path = '/mnt/scratch/erick/ffcei/papersim/sequential_instability/seq_ins_1xem/slow_left_4p' # QKE simulation 1xem
directory_path = '/mnt/scratch/erick/ffcei/papersim/sequential_instability/seq_ins_8xem/slow_left_4p' # QKE simulation 8xem

file_list = glob.glob(directory_path + '/plt*.h5')
file_list.sort(key=lambda x: int(x.split('plt')[1].split('.h5')[0]))

In [ ]:
def read_param_file(filename):
    def set_nested(d, k, v):
        keys = k.split('.')
        for key in keys[:-1]:
            d = d.setdefault(key, {})
        d[keys[-1]] = v

    params = {}
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#') or '=' not in line:
                continue
            key, val = map(str.strip, line.split('=', 1))
            set_nested(params, key, ast.literal_eval(val))
    return params

# Usage
params = read_param_file(directory_path+"/inputs_fast_flavor_nonzerok")

In [ ]:
# Domain size in 3D index space
ncell = params['ncell'] # Number of cells in x, y, z

Lx = params['Lx'] # Domain size in x (cm)
Ly = params['Ly'] # Domain size in y (cm)
Lz = params['Lz'] # Domain size in z (cm)

dx = Lx/ncell[0] #cm
dy = Ly/ncell[1] #cm
dz = Lz/ncell[2] #cm

cellvolume = dx*dy*dz # ccm

In [ ]:
# ['N00_Re', 'N00_Rebar', 
# 'N01_Im', 'N01_Imbar', 
# 'N01_Re', 'N01_Rebar', 
# 'N11_Re', 'N11_Rebar',
# 'TrHN', 'Vphase',
# 'pos_x', 'pos_y', 'pos_z',
# 'pupt', 'pupx', 'pupy', 'pupz',
# 'time', 'x', 'y', 'z']

time_seconds = np.zeros(len(file_list))
Pz_pzpos_av  = np.zeros(len(file_list))
Pz_pzneg_av  = np.zeros(len(file_list))
Pz_pxpos_av  = np.zeros(len(file_list))
Pz_pxneg_av  = np.zeros(len(file_list))

N_pzpos_av  = np.zeros((len(file_list),2,2))
N_pzneg_av  = np.zeros((len(file_list),2,2))
N_pxpos_av  = np.zeros((len(file_list),2,2))
N_pxneg_av  = np.zeros((len(file_list),2,2))


for i, file_name in enumerate(file_list):

    with h5py.File(file_name, 'r') as f:
        data_dict = {}
        for key in f.keys():
            data_dict[key] = np.array(f[key][:])

    px = np.array(data_dict['pupx']) 
    py = np.array(data_dict['pupy'])
    pz = np.array(data_dict['pupz'])

    mask_pz_pos = (pz>0) & (px==0)
    mask_pz_neg = (pz<0) & (px==0)
    mask_px_pos = (px>0) & (pz==0)
    mask_px_neg = (px<0) & (pz==0)

    z = np.array(data_dict['z'])

    Nee   = np.array(data_dict['N00_Re'])
    Nuu   = np.array(data_dict['N11_Re'])
    NeuRe = np.array(data_dict['N01_Re'])
    NeuIm = np.array(data_dict['N01_Im'])

    if i == 0:
        print('Initial')
        Ntotal = np.sum(Nee + Nuu)
        print('Ntotal:', Ntotal)
        ELNXLN = np.sum(Nee - Nuu)
        print('ELNXLN:', ELNXLN)
        print('')
    if i == len(file_list)-1:
        print('Final')
        Ntotal = np.sum(Nee + Nuu)
        print('Ntotal:', Ntotal)
        ELNXLN = np.sum(Nee - Nuu)
        print('ELNXLN:', ELNXLN)
        print('')

    Pz = 0.5 * ( Nee - Nuu )
    Px = NeuRe
    Py = NeuIm

    time_seconds[i] = data_dict['time'][0]
    Pz_pzpos_av[i]  = np.average(Pz[mask_pz_pos]) / cellvolume
    Pz_pzneg_av[i]  = np.average(Pz[mask_pz_neg]) / cellvolume
    Pz_pxpos_av[i]  = np.average(Pz[mask_px_pos]) / cellvolume 
    Pz_pxneg_av[i]  = np.average(Pz[mask_px_neg]) / cellvolume

    N_pzpos_av[i,0,0] = np.average(np.array(data_dict['N00_Re'])[mask_pz_pos])
    N_pzpos_av[i,1,1] = np.average(np.array(data_dict['N11_Re'])[mask_pz_pos])
    N_pzpos_av[i,0,1] = np.average(np.sqrt(np.array(data_dict['N01_Re'])[mask_pz_pos]**2+np.array(data_dict['N01_Im'])[mask_pz_pos]))

    N_pzneg_av[i,0,0] = np.average(np.array(data_dict['N00_Re'])[mask_pz_neg])
    N_pzneg_av[i,1,1] = np.average(np.array(data_dict['N11_Re'])[mask_pz_neg])
    N_pzneg_av[i,0,1] = np.average(np.sqrt(np.array(data_dict['N01_Re'])[mask_pz_neg]**2+np.array(data_dict['N01_Im'])[mask_pz_neg]))

    N_pxpos_av[i,0,0] = np.average(np.array(data_dict['N00_Re'])[mask_px_pos])
    N_pxpos_av[i,1,1] = np.average(np.array(data_dict['N11_Re'])[mask_px_pos])
    N_pxpos_av[i,0,1] = np.average(np.sqrt(np.array(data_dict['N01_Re'])[mask_px_pos]**2+np.array(data_dict['N01_Im'])[mask_px_pos]))

    N_pxneg_av[i,0,0] = np.average(np.array(data_dict['N00_Re'])[mask_px_neg])
    N_pxneg_av[i,1,1] = np.average(np.array(data_dict['N11_Re'])[mask_px_neg])
    N_pxneg_av[i,0,1] = np.average(np.sqrt(np.array(data_dict['N01_Re'])[mask_px_neg]**2+np.array(data_dict['N01_Im'])[mask_px_neg]))   

In [ ]:
nee_pz_pos_init = 2.4456e+32 # 1/ccm
nuu_pz_neg_init = 2.4456e+32 # 1/ccm

nee_pzpos = 4.8913e+32 # 1/ccm
nxx_pzpos = 0.0 # 1/ccm

nee_pzneg = 0.0 # 1/ccm
nxx_pzneg = 2.4456e+32 # 1/ccm

nee_pxpos = 2.4456e+32 # 1/ccm
nxx_pxpos = 0.0 # 1/ccm

nee_pxneg = 2.4456e+32 # 1/ccm
nxx_pxneg = 0.0 # 1/ccm

Pz_pz_pos_init = 0.5 * nee_pzpos # 1/ccm
# Pz_pz_neg_init = nuu_pz_neg_init # 1/ccm

mu_t = 1.0 / (np.array(np.sqrt(2)*PhysConst.GF*(nee_pzpos+nxx_pzpos+nee_pzneg+nxx_pzneg+nee_pxpos+nxx_pxpos+nee_pxneg+nxx_pxneg))*(PhysConst.c**3*PhysConst.hbar**2)) # seconds
mu_l = 1.0 / (np.array(np.sqrt(2)*PhysConst.GF*(nee_pzpos+nxx_pzpos+nee_pzneg+nxx_pzneg+nee_pxpos+nxx_pxpos+nee_pxneg+nxx_pxneg))*(PhysConst.c**2*PhysConst.hbar**2)) # cm

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, Pz_pzpos_av, label = r'$\hat{\mathbf{p}}=(0,0,1)$', color='C0', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, Pz_pzneg_av,  label=r'$\hat{\mathbf{p}}=(0,0,-1)$', color='C0', linestyle='solid', linewidth=3)
ax.plot(time_seconds, Pz_pxpos_av, label = r'$\hat{\mathbf{p}}=(1,0,0)$', color='C0', linestyle='dotted', linewidth=3)
ax.plot(time_seconds, Pz_pxneg_av, label = r'$\hat{\mathbf{p}}=(-1,0,0)$', color='C0', linestyle='dotted', linewidth=3)
# ax.axhline(y= Pz_pz_pos_init, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=-Pz_pz_pos_init, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y= Pz_pz_neg_init, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=-Pz_pz_neg_init, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=0.0, color='gray', linestyle='--', linewidth=1)
# ax.axvline(x=1.698836375440867e-09, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<P_z\right> \, [\mathrm{cm}^{-3}]$')
# ax.set_yscale('log')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=20)
apply_custom_settings(ax, leg, False)
# plt.savefig(directory_path+"/nee_angular_distribution.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(time_seconds/mu_t, Pz_pzpos_av/Pz_pz_pos_init, label = r'$\hat{\mathbf{p}}=(0,0,1)$', color='C0', linestyle='dashed', linewidth=3)
ax.plot(time_seconds/mu_t, Pz_pzneg_av/Pz_pz_pos_init,  label=r'$\hat{\mathbf{p}}=(0,0,-1)$', color='C0', linestyle='solid', linewidth=3)
ax.plot(time_seconds/mu_t, Pz_pxpos_av/Pz_pz_pos_init, label = r'$\hat{\mathbf{p}}=(1,0,0)$', color='C0', linestyle='dotted', linewidth=3)
ax.plot(time_seconds/mu_t, Pz_pxneg_av/Pz_pz_pos_init, label = r'$\hat{\mathbf{p}}=(-1,0,0)$', color='C0', linestyle='dotted', linewidth=3)

Total_Pz = (Pz_pzpos_av + Pz_pzneg_av + Pz_pxpos_av + Pz_pxneg_av)/Pz_pz_pos_init
print("Initial Total_Pz:", Total_Pz[0])
print("Final Total_Pz:", Total_Pz[-1])

print(f'time_mu_{directory_path.split("/")[-1]}={list(time_seconds/mu_t)}')
print(f'Pz_pzpos_av_{directory_path.split("/")[-1]}={list(Pz_pzpos_av/Pz_pz_pos_init)}')
print(f'Pz_pzneg_av_{directory_path.split("/")[-1]}={list(Pz_pzneg_av/Pz_pz_pos_init)}')
print(f'Pz_pxpos_av_{directory_path.split("/")[-1]}={list(Pz_pxpos_av/Pz_pz_pos_init)}')
print(f'Pz_pxneg_av_{directory_path.split("/")[-1]}={list(Pz_pxneg_av/Pz_pz_pos_init)}')

# ax.axhline(y=0, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=1, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=-1, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=1/2, color='gray', linestyle='--', linewidth=1)
# ax.axhline(y=-1/2, color='gray', linestyle='--', linewidth=1)
# ax.axvline(x=2400, color='gray', linestyle='--', linewidth=1)
# ax.axvline(x=2*2400, color='gray', linestyle='--', linewidth=1)

ax.set_xlabel(r'$t$ ($\mu^{-1}$)')
ax.set_ylabel(r'$\left<P_z\right>$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=20)
apply_custom_settings(ax, leg, False)
plt.savefig("./plots/Pz_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, N_pzpos_av[:,0,1], label = r'$\hat{\mathbf{p}}=+\hat{z}$', color='C0', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pzneg_av[:,0,1], label = r'$\hat{\mathbf{p}}=-\hat{z}$', color='C1', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxpos_av[:,0,1], label = r'$\hat{\mathbf{p}}=+\hat{x}$', color='C2', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxneg_av[:,0,1], label = r'$\hat{\mathbf{p}}=-\hat{x}$', color='C2', linestyle='solid', linewidth=3)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<N_{ex}\right>$')
# leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18, loc='center left', bbox_to_anchor=(1, 0.5))
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18)
# ax.set_xlim(0, 0.5e-9)
ax.set_ylim(1e29, None)
apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig("./plots/N_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, N_pzpos_av[:,0,0], label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(0,0,1)$', color='C0', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pzneg_av[:,0,0], label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(0,0,-1)$', color='C1', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxpos_av[:,0,0], label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(1,0,0)$', color='C2', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxneg_av[:,0,0], label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(-1,0,0)$', color='C2', linestyle='solid', linewidth=3)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<N_{ee}\right>$')
# leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18, loc='center left', bbox_to_anchor=(1, 0.5))
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18)
# ax.set_xlim(0, 0.5e-9)
ax.set_ylim(1e29, None)
apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig("./plots/N_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, N_pzpos_av[:,1,1], label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(0,0,1)$', color='C0', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, N_pzneg_av[:,1,1], label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(0,0,-1)$', color='C1', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, N_pxpos_av[:,1,1], label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(1,0,0)$', color='C2', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, N_pxneg_av[:,1,1], label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(-1,0,0)$', color='C2', linestyle='dashed', linewidth=3)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<N_{xx}\right>$')
# leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18, loc='center left', bbox_to_anchor=(1, 0.5))
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18)
# ax.set_xlim(0, 0.5e-9)
ax.set_ylim(1e29, None)
apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig("./plots/N_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, N_pzpos_av[:,0,1] / ( N_pzpos_av[:,0,0] + N_pzpos_av[:,1,1] ), label = r'$\hat{\mathbf{p}}=+\hat{z}$', color='C0', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pzneg_av[:,0,1] / ( N_pzneg_av[:,0,0] + N_pzneg_av[:,1,1] ), label = r'$\hat{\mathbf{p}}=-\hat{z}$', color='C1', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxpos_av[:,0,1] / ( N_pxpos_av[:,0,0] + N_pxpos_av[:,1,1] ), label = r'$\hat{\mathbf{p}}=+\hat{x}$', color='C2', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxneg_av[:,0,1] / ( N_pxneg_av[:,0,0] + N_pxneg_av[:,1,1] ), label = r'$\hat{\mathbf{p}}=-\hat{x}$', color='C2', linestyle='solid', linewidth=3)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<\rho_{ex}\right>$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18)
# ax.set_xlim(0, 0.5e-9)
ax.set_ylim(1e-2, 4e-1)
apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig("./plots/N_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, N_pzpos_av[:,0,0] / ( N_pzpos_av[:,0,0] + N_pzpos_av[:,1,1] ), label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(0,0,1)$', color='C0', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pzneg_av[:,0,0] / ( N_pzneg_av[:,0,0] + N_pzneg_av[:,1,1] ), label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(0,0,-1)$', color='C1', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxpos_av[:,0,0] / ( N_pxpos_av[:,0,0] + N_pxpos_av[:,1,1] ), label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(1,0,0)$', color='C2', linestyle='solid', linewidth=3)
ax.plot(time_seconds, N_pxneg_av[:,0,0] / ( N_pxneg_av[:,0,0] + N_pxneg_av[:,1,1] ), label = r'$N_{ee}\,:\, \hat{\mathbf{p}}=(-1,0,0)$', color='C2', linestyle='solid', linewidth=3)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<\rho_{ee}\right>$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18)
# ax.set_xlim(0, 0.5e-9)
ax.set_ylim(1e-2, 1.1)
apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig("./plots/N_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_seconds, N_pzpos_av[:,1,1] / (N_pzpos_av[:,0,0] + N_pzpos_av[:,1,1]), label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(0,0,1)$', color='C0', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, N_pzneg_av[:,1,1] / (N_pzneg_av[:,0,0] + N_pzneg_av[:,1,1]), label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(0,0,-1)$', color='C1', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, N_pxpos_av[:,1,1] / (N_pxpos_av[:,0,0] + N_pxpos_av[:,1,1]), label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(1,0,0)$', color='C2', linestyle='dashed', linewidth=3)
ax.plot(time_seconds, N_pxneg_av[:,1,1] / (N_pxneg_av[:,0,0] + N_pxneg_av[:,1,1]), label = r'$N_{xx}\,:\, \hat{\mathbf{p}}=(-1,0,0)$', color='C2', linestyle='dashed', linewidth=3)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$\left<\rho_{xx}\right>$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=18)
# ax.set_xlim(0, 0.5e-9)
ax.set_ylim(1e-2, 1.1)
apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig("./plots/N_"+directory_path.split('/')[1]+".pdf", bbox_inches='tight')
plt.show()
plt.close(fig)